## Label image data in plate 180528_Plate3
*Vladislav Kim*


* [Introduction](#1)
* [Initial training set](#2)

<a id="1"></a> 
## Introduction
The idea of this notebook series is to train a pseudo-online random forest classifier for AML vs stroma cell classification. From selected plates we (for now) sample 6 DMSO wells with the highest Calcein cell count, generate predictions and correct misclassified instances and check in live ("online") mode how the predictions improve as we add more data. Note that the classifier is not truly an online classifier as we don't update the model as we go, but completely retrain the RF classfier in multicore mode.

In general we can implement targeted online learning strategy: we can select a number of wells that are of interest to us (target wells), e.g. DMSO control wells or wells with certain high-priority drugs, the accuracy of which we want to improve, in the first place. We sample from these target wells from selected plates and evaluate the classification accuracy as we go (pseudo online learning).


<a id="2"></a>
## Initial Training Set: 180528_Plate3
At first we will re-train the classifier on the plate `180528_Plate3`, as it manifests a very striking contrast between mono- and co-cultures. We want to rule out the fact that this could be a segmentation (classification in this case) artefact 

In [ ]:
# load third-party Python modules
import javabridge
import bioformats as bf
import skimage
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import os
import sys
sys.path.append('../../..')

javabridge.start_vm(class_path=bf.JARS)

In [ ]:
from base.utils import load_imgstack
imgstack = load_imgstack(fname="../../data/AML_trainset/180528_Plate3/r02c14.tiff")

# remove a 'dummy' z-axis
img = np.squeeze(imgstack)

# nuclei
hoechst = img[:,:,0]**0.3

In [ ]:
df = pd.read_csv('../../data/AML_trainset/180528_Plate3/r02c14.csv')

In [ ]:
from segment.tools import read_bbox
rmax, cmax = hoechst.shape

bbox = read_bbox(df=df, rmax=rmax, cmax=cmax)

In [ ]:
from base.plot import show_bbox
#show_bbox(hoechst, bbox)

**Plotly visualization works!**

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [ ]:
from extra.viz import plotly_viz

In [ ]:
from skimage.exposure import equalize_adapthist
gamma = 0.3
img_g = img**gamma
mip_rgb = equalize_adapthist(np.dstack((img_g[:,:,1],
                                        img_g[:,:,2],
                                        img_g[:,:,0])))

In [ ]:
layout, cells = plotly_viz(mip_rgb, bb=bbox)

In [ ]:
from extra.viz import plotly_predictions
ypred = np.zeros(len(bbox), dtype=np.int)
labels = ['cells']
layout, cells = plotly_predictions(img=mip_rgb, bb=bbox,
                                  ypred=ypred, labels=labels)

In [ ]:
#iplot(dict(data=cells, layout=layout))

In [ ]:
def get_train_instance(path, fname, pad=0):
    imgstack = load_imgstack(fname=os.path.join(path, fname + ".tiff"),
                            verbose=False)
    img = np.squeeze(imgstack)
    df = pd.read_csv(os.path.join(path, fname + ".csv"))
    rmax, cmax, _ = img.shape
    bbox = read_bbox(df=df, rmax=rmax,
                     cmax=cmax, pad=pad)
    return img, bbox

In [ ]:
img, bbox = get_train_instance(path='../../data/AML_trainset/180528_Plate3',
                              fname='r02c14', pad=20)

**Here need to fix how we import `ImgX`**

In [ ]:
sys.path.append('../../../../')
from bioimg.classify import ImgX

In [ ]:
imgx_test = ImgX(img=img**0.4, bbox=bbox, n_chan=3)
imgx_test = imgx_test.compute_props()

In [ ]:
# imgx_test.data[0].iloc[:10,:12]

**Implement a class that has `ImgX` instance and can accumulate training data:**
+ Should we initialize the classifier from within the class or should users pass the classifier to `IncrementalClassifier` constructor?

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier


class IncrementalClassifier:
    def __init__(self):
        # initialize with 'None' something to be loaded later
        self.imgx = None
        
        # for the plotly visualization
        self.feats = None
        self.layout = None
        
        self.newlabels = None
        # training data
        self.Xtrain = None
        self.ytrain = None
        # inialize classifier as 'None'
        self.clf = None
        self.classes = None

    def __setattr__(self, name, value):
        self.__dict__[name] = value
        # if a new ImgX object is passed,
        # compute its features
        if name == 'imgx':
            self._compute_imgx_data()

    # function for setting individual class parameters
    def set_param(self, **kwargs):
        for k in kwargs.keys():
            self.__setattr__(k, kwargs[k])
         
    # internal function checks if the embedded
    # imgx object has the features computed
    def _compute_imgx_data(self):
        if self.imgx is not None and len(self.imgx.data) == 0:
            self.imgx.compute_props()
            
    # set plotly graphical layers
    def set_scene(self):
        if self.imgx.y is not None:
            self.layout, self.feats = plotly_predictions(img=self.imgx.img,
                                                         bb=self.imgx.bbox,
                                                         ypred=self.imgx.y
                                                        )
        else:
            self.layout, self.feats = plotly_viz(img=self.imgx.img,
                                                    bb=self.imgx.bbox)
        return self

    # plot predictions overlaid with the original image
    # plot is a 'void' function (returns 'None')
    def plot(self):
        iplot(dict(data=self.feats, layout=self.layout))
        
    def add_instances(self, newlabels):
        newlabels = np.unique(newlabels, axis=0)
        if self.newlabels is None:
            self.newlabels = newlabels
        else:
            a1_rows = newlabels.view(
                [('', newlabels.dtype)] * newlabels.shape[1])
            a2_rows = self.newlabels.view(
                [('', self.newlabels.dtype)] * self.newlabels.shape[1])

            newlabels = (np.setdiff1d(a1_rows, a2_rows).
                         view(newlabels.dtype).
                         reshape(-1, newlabels.shape[1]))
            self.newlabels = np.append(self.newlabels, newlabels, axis=0)
        return self
    
    def _get_traindata(self):
        ids = self.newlabels[:,0]
        data_dict = self.imgx.data
        for k,v in data_dict.items():
            v.columns = ['-'.join(['ch', str(k), col]) for col in v.columns.values]
        data = pd.concat(data_dict, axis=1)
        data.columns = data.columns.droplevel()
        
        self.Xtrain = data.iloc[ids,:]
        self.ytrain = label_binarize(self.newlabels[:,1], classes=self.classes)
        
    def set_classifier(self, clf=None):
            self.clf = clf
            # if 'None' then some reasonable default
            if clf is None:
                self.clf = OneVsRestClassifier(RandomForestClassifier(bootstrap=True,
                                                  class_weight="balanced",
                                                  n_estimators=500,
                                                  random_state=123,
                                                  n_jobs=-1))
            return self
    
    def train_classifier(self):
        self._get_traindata()
        self.clf.fit(self.Xtrain, self.ytrain)
        return self
        

**Sort the `img` array so that the color channel order is RGB**

In [ ]:
gamma = 0.4
# adjust brightness by gamma correction
img_g = img**gamma
# sort color channels in'RGB' order
img_rgb = np.dstack((img_g[:,:,1],
                     img_g[:,:,2],
                     img_g[:,:,0]))
# initialize 'ImgX' class
imgx = ImgX(img=img_rgb, bbox=bbox, n_chan=3)

In [ ]:
clf_incr = IncrementalClassifier()

In [ ]:
clf_incr.imgx = imgx

In [ ]:
#clf_incr.set_scene().plot()

In [ ]:
newlabels = np.array([[43,1], [48,1], [29,1],
                      [55,1], [123,0], [76,0], 
                      [82,0], [82,0]])

In [ ]:
clf_incr = clf_incr.add_instances(newlabels=newlabels)

**So `_get_traindata` works apparently, call it before `train_classifier`**

In [ ]:
clf_incr._get_traindata()

**Add `OneVsRestClassifier` and binarize the target variable `y`**

In [ ]:
# generate predictions with the loaded feature data
def generate_predictions(self, prob=False):
    if self.X_test_norm is None:
        cellbb_norm = [resize(cb, (self.w, self.h), anti_aliasing=True)
                       for cb in self.cellbb]
        self.X_test_norm = np.array([cbn.ravel() for cbn in cellbb_norm])

    if self.X_test_prop is None:
        X_prop_list = [get_regionprop_feats(
            mip_rgb=cbb, exclude=exclude) for cbb in self.cellbb]
        self.X_test_prop = np.vstack(X_prop_list)

    X_test_pca = self.pca.transform(self.X_test_norm)
    X_test_all = np.append(X_test_pca, self.X_test_prop, axis=1)

    self.y_pred = self.clf.predict(X_test_all)
    if prob:
        self.y_prob = self.clf.predict_proba(X_test_all)
    return self


# print the confusion matrix on the existing training set
def get_classifiction_report(self):
    X_train_pca = self.pca.transform(self.X_train_norm)
    X_train_all = np.append(X_train_pca, self.X_train_prop, axis=1)
    y_pred = self.clf.predict(X_train_all)
    print(classification_report(self.y_train,
                                y_pred, target_names=self.target_names))
    print(confusion_matrix(self.y_train, y_pred,
                           labels=range(len(self.target_names))))
    return self

def get_cross_val_score(self, kfold=5):
    X_train_pca = self.pca.transform(self.X_train_norm)
    X_train_all = np.append(X_train_pca, self.X_train_prop, axis=1)
    scores = cross_val_score(self.clf, X_train_all, self.y_train, cv=kfold)
    print scores

    return self

def reset(self):
    self.newlabels = None
    self.X_test_norm = None
    self.X_test_prop = None
    return self

def h5_write(self, fname):
    hf = h5py.File(fname, 'w')
    hf.create_dataset('X_train_norm', data=self.X_train_norm)
    hf.create_dataset('X_train_prop', data=self.X_train_prop)
    hf.create_dataset('y_train', data=self.y_train)
    hf.close()



In [ ]:
'''# old version of the function
def compute_props(self):
        X_prop_list =  [OT.get_regionprop_feats(mip_rgb=cbb,
                                                exclude=exclude) for cbb in cellbb_train]
        X_train_prop = np.vstack(X_prop_list)


        cellbb_norm = [resize(cb, (w, h), anti_aliasing=True) for cb in cellbb_train]
        X_train_norm = np.array([cbn.ravel() for cbn in cellbb_norm])
        # compute PCA of the image data set
        n_components = 150
        pca = PCA(n_components=n_components, svd_solver='randomized',
                  whiten=True).fit(X_train_norm)

        # project the train data
        X_train_pca = pca.transform(X_train_norm)

        X_train_all = np.append(X_train_pca, X_train_prop, axis=1)
        
        return self'''

**Modify `IncrementalClassifier` class to adapt to our use**

In [ ]:
# incremental ("online") classifier
clf_incr = OT.IncrementalClassifier(path=path, featdir=featdir,
                                 select_well=select_inst[0],
                                 target_names=target_names,
                                 X_train_norm=X_train_norm,
                                 X_train_prop=X_train_prop,
                                 y_train=y_train
                                )

In [ ]:
clf_incr = (clf_incr.load_img().
            train_classifier().
           generate_predictions().
           set_scene())

In [ ]:
#clf_incr.plot()

In [ ]:
newlabels = np.array([[45,2], [91,5], [85,0], [2,2]])

In [ ]:
clf_incr = (clf_incr.
            add_instances(newlabels=newlabels).
            train_classifier().
            generate_predictions().update_scene())